In [2]:
from bs4 import BeautifulSoup
import requests
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time

import re
import numpy as np
import pandas as pd

#create dataframe
df=pd.DataFrame()

#source URL
URL= "https://www.hktvmall.com/hktv/en/main/search?page=0&q=%3Arelevance%3Acategory%3AAA16107500000%3Azone%3Abeautynhealth%3Astreet%3Amain%3A"
URL1="https://www.hktvmall.com/hktv/en/main/search?page="
URL2="&q=%3Arelevance%3Acategory%3AAA16107500000%3Azone%3Abeautynhealth%3Astreet%3Amain%3A"

#run selenium wbdriver
driver = webdriver.Chrome(executable_path="/Applications/chromedriver")
driver.get(URL)
time.sleep(5)

html = driver.page_source
soup=BeautifulSoup(html, "html.parser")
URL_iter =0

#extract information
while int(URL_iter)<9:
    for item in soup.find_all(class_="info-wrapper"):
        
        #Raw info
        try:
            p_raw=item.find(class_="brand-product-name").text
            print("Raw Info:", p_raw)
        except:
            p_raw=str(np.nan)
            print("Raw Info:", p_raw)
        #Brand name
        try:
            p_brand=re.match("^(.*?)\s\-\s", p_raw).group(1)
            print("Product Brand:", p_brand)
        except:
            p_brand=p_raw
            print("Product Brand:", p_brand)
        #Product Name
        try:
            p_item=re.match("^(?:.*)\s\-\s(.*)", p_raw).group(1)
            print("Product Name:", p_item)

        except:
            p_item=p_raw
            print("Product Name:", p_item)
        #No. of Sales
        try:
            p_sales=item.find(class_="salesNumber-container").text.replace("+ Sold", "")
            print("Sales:", p_sales)
        except:
            p_sales=np.nan
            print("Sales:", p_sales)
        #Original price
        try:
            p_oprice=item.find(class_="promotional").span.text
            p_oprice2=re.match("^(?:.*\$)(.*)", p_oprice).group(1)
            print("Original Price:", p_oprice2)
        except:
            p_oprice=np.nan 
            print("Original Price:", p_oprice)       
        #Discounted Price
        try:
            p_dprice=item.find(class_="price").span.text.replace("$ ", "")
            print("Discount Price:", p_dprice)
        except:
            p_dprice=np.nan
            print("Discount Price:", p_dprice)
        #Promotional tag
        try:
            promo=item.find(class_="deliveryLabelText").text
            print("Promo Tag:", promo)
        except:
            promo=np.nan
            print("Promo Tag:", promo)
        #Product Rating
        try:
            rating=item.find(class_="star_container")["data-rating"]
            print("Rating:", rating)
        except:
            rating=np.nan
            print("Rating:", rating)
        #No. of Reviews
        try:
            review=item.find(class_="review-number").text.replace("(", "").replace(")", "")
            print("Review:", review)
        except:
            review=np.nan
            print("Review:", review)
        #Vendor
        try:
            vendor=item.find(class_="store-name-label").text
            print("Vendor:", vendor)
        except:
            vendor=np.nan
            print("Vendor:", vendor)
        #Category code
        try:
            category=item.find(class_="sepaButton add-to-cart-button")["data-category"]
            print("Product Category Code:", category)
        except:
            category=str(np.nan)
            print("Product Category Code:", category)

        print("\n")
        #save to dataframe
        df=df.append({"Raw Product":p_raw, "Product Brand":p_brand, "Product Name":p_item, "Sales":p_sales, "Original Price":p_oprice2, "Discount price":p_dprice, "Promo Tag":promo, "Rating":rating, "Review":review, "Vendor":vendor, "Product Category Code":category}, ignore_index=True)
    
    #load next page
    URL_no=re.findall("\?page\=(\d*)\&q\=.*",URL)
    URL_iter=str(int(URL_no[0])+1)

    URL = URL1+URL_iter+URL2
    nextlink=URL1+URL_iter+URL2
    print(nextlink)

    subhtml=driver.get(nextlink)
    time.sleep(5)
    subhtml=driver.page_source
    soup=BeautifulSoup(subhtml, "html.parser")
#save dataframe as csv
df.to_csv("hktvmall_skincare_males.csv")


Vendor: HAIR FOREST
Product Category Code: AA13203520001


Raw Info: Shiseido - Men Cleansing Foam 125ml
Product Brand: Shiseido
Product Name: Men Cleansing Foam 125ml
Sales: nan
Original Price:  300.00
Discount Price: 228.00
Promo Tag: 2-day group shipment
Rating: nan
Review: nan
Vendor: Beaute House
Product Category Code: AA16107510001


Raw Info: SK-II - SK2/SKII Men Moisturizing Cleanser 120g
Product Brand: SK-II
Product Name: SK2/SKII Men Moisturizing Cleanser 120g
Sales: nan
Original Price:  680.00
Discount Price: 498.00
Promo Tag: 2-day group shipment
Rating: nan
Review: nan
Vendor: Beaute House
Product Category Code: AA16107510001


Raw Info: Shiseido - MEN HYDRATING LOTION 150ML
Product Brand: Shiseido
Product Name: MEN HYDRATING LOTION 150ML
Sales: nan
Original Price:  283.00
Discount Price: 198.00
Promo Tag: 2-day group shipment
Rating: nan
Review: nan
Vendor: The One Baby
Product Category Code: AA16107520001


Raw Info: Shiseido - CLARIFYING CLEANSING FOAM FOR ALL SKIN TYP

In [3]:
df.shape

(533, 11)

In [6]:
df.to_csv("hktvmall_skincare_males.csv")

In [11]:
df.dtypes

Discount price           object
Original Price           object
Product Brand            object
Product Category Code    object
Product Name             object
Promo Tag                object
Rating                   object
Raw Product              object
Review                   object
Sales                    object
Vendor                   object
dtype: object